In [ ]:
import sys

sys.path.append('../src')
sys.path.append('../tests')

In [ ]:
import os
import gc
import bcolz
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History
import vgg16
import data_helper
from data_helper import AmazonPreprocessor
from PIL import Image
# from kaggle_data.downloader import KaggleDataDownloader

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
train_jpeg_dir, test_jpeg_dir, test_jpeg_additional, train_csv_file = data_helper.get_jpeg_data_files_paths()
labels_df = pd.read_csv(train_csv_file)
labels_df.head()

In [ ]:
# Print all unique tags
from itertools import chain
labels_list = list(chain.from_iterable([tags.split(" ") for tags in labels_df['tags'].values]))
labels_set = set(labels_list)
print("There is {} unique labels including {}".format(len(labels_set), labels_set))

In [ ]:
# Histogram of label instances
labels_s = pd.Series(labels_list).value_counts() # To sort them by count
fig, ax = plt.subplots(figsize=(16, 8))
sns.barplot(x=labels_s, y=labels_s.index, orient='h')


In [ ]:
labels_df.columns

In [ ]:
artisianmining = []
primary = []
clear = []
slash_burn = []
agriculture = []
train_jpeg_dir 
total = 0

for index, row in labels_df.iterrows():
    tags = row["tags"].split()
    
    for tag in tags:
        if (tag == "artisinal_mine"):
            total += 1
            artisianmining.append(row["image_name"])
        if (tag == "primary"):
            primary.append(row["image_name"])
            
        if (tag == "slash_burn"):
            slash_burn.append(row["image_name"])
            
        if (tag == "clear"):
            clear.append(row["image_name"])
        
        if (tag == "agriculture"):
            agriculture.append(row["image_name"])
    

In [ ]:
def crop(im,height,width):
    # im = Image.open(infile)
    imgwidth, imgheight = im.size
    for i in range(int(imgheight//height)):
        for j in range(int(imgwidth//width)):
            # print (i,j)
            box = (j*width, i*height, (j+1)*width, (i+1)*height)
            yield im.crop(box)

In [ ]:
# metadata = pd.DataFrame(columns = ["newimage", "oldimage", "class"])
metadata = pd.read_csv("metadata.csv")

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# image_slicer.slice(image_location, 4)

In [ ]:
# imgplot = plt.imshow(im)
# plt.show()

In [ ]:
# Histogram of label instances
labels_s = pd.Series(labels_list).value_counts() # To sort them by count
fig, ax = plt.subplots(figsize=(16, 8))
sns.barplot(x=labels_s, y=labels_s.index, orient='h')

In [ ]:
def relabel_unknown():
    class_dict = {1: "primary", 2: "clear", 3: "artisinal_mine", 
                  4: "agriculture", 5: "road", 6: "water", 7: "partly_cloudy", 
                  8: "cloudy", 9: "habitation", 10: "haze", 
                  11: "bare_ground", 12: "selective_logging", 13: "blooming", 
                  14: "slash_burn", 15: "blow_down", 16: "conventional_mine", 
                  17: "cultivation"}
    
    
    for image in artisianmining:
        image_location = train_jpeg_dir + "/" + image + ".jpg"
        im = Image.open(image_location)
        
        quadrant = 1
        metadata = pd.read_csv("metadata.csv")
        if image in metadata["oldimage"].values:
            print("seen image: continuing....")
            continue
        
        print("unseen image: ", image)
        
        for i in range(0, 256, 128):
            for j in range(0, 256, 128):
                box = (int(i), int(j), int(i + 128), int(j + 128))
                croppedimage = im.crop(box)

                dest = "new_images/" + image + "_" + str(quadrant) + ".jpg"
                croppedimage.resize((128, 128))
                croppedimage.save(dest)
                quadrant += 1
                imgplot = plt.imshow(croppedimage)
                plt.show()
                print("codes: ", class_dict)
                class_num = int(input())
                row = {"newimage": image + "_" + str(quadrant), "oldimage": image, "class": class_dict[class_num]}
                print(row)
        
                metadata = metadata.append(row, ignore_index = True)
                metadata.to_csv("metadata.csv", index = False)
                print("adding row to metadata.csv")


In [ ]:
relabel_unknown()

In [ ]:
def relabel_primary():
    class_dict = {1: "primary", 2: "clear", 3: "artisinal_mine", 
                  4: "agriculture", 5: "road", 6: "water", 7: "partly_cloudy", 
                  8: "cloudy", 9: "habitation", 10: "haze", 
                  11: "bare_ground", 12: "selective_logging", 13: "blooming", 
                  14: "slash_burn", 15: "blow_down", 16: "conventional_mine", 
                  17: "cultivation"}
    
    
    for image in primary:
        image_location = train_jpeg_dir + "/" + image + ".jpg"
        im = Image.open(image_location)
        
        quadrant = 1
        metadata = pd.read_csv("metadata.csv")
        if image in metadata["oldimage"].values:
            print("seen image: continuing....")
            continue
        
        print("unseen image: ", image)
        
        for i in range(0, 256, 128):
            for j in range(0, 256, 128):
                box = (int(i), int(j), int(i + 128), int(j + 128))
                croppedimage = im.crop(box)

                dest = "new_images/" + image + "_" + str(quadrant) + ".jpg"
                croppedimage.resize((128, 128))
                croppedimage.save(dest)
                quadrant += 1
                imgplot = plt.imshow(croppedimage)
                plt.show()
                print("codes: ", class_dict)
                class_num = int(input())
                row = {"newimage": image + "_" + str(quadrant), "oldimage": image, "class": class_dict[class_num]}
                print(row)
        
                metadata = metadata.append(row, ignore_index = True)
                metadata.to_csv("metadata.csv", index = False)
                print("adding row to metadata.csv")


In [ ]:
relabel_primary()

In [ ]:
metadata = pd.read_csv("metadata.csv")

In [ ]:
metadata

In [ ]:
# image = "train_57"
# image_location = train_jpeg_dir + "/" + image + ".jpg"
# im = Image.open(image_location)
# imgwidth, imgheight = im.size
# print('Image size is: %d x %d ' % (imgwidth, imgheight))
# # height = int(imgheight/2)
# # width =  int(imgwidth/2)
# # start_num = 0


# class_dict = {1: "primary", 2: "clear", 3: "artisinal_mine"}
# quadrant = 1
# for i in range(0, 256, 128):
#     for j in range(0, 256, 128):
#         box = (int(i), int(j), int(i + 128), int(j + 128))
#         print(box)
#         croppedimage = im.crop(box)
        
#         dest = "new_images/" + image + "_" + str(quadrant) + ".jpg"
# #         croppedimage.convert('RGB')
#         croppedimage.resize((128, 128))
#         croppedimage.save(dest)
#         quadrant += 1
#         imgplot = plt.imshow(croppedimage)
#         plt.show()
        
#         class_num = int(input())
#         row = {"newimage": image + "_" + str(quadrant), "oldimage": image, "class": class_dict[class_num]}
#         print(row)
        
        
        

        

# # for k,piece in enumerate(crop(im,height,width),start_num):
# #     # print k
# #     # print piece
# #     img=Image.new('RGB', (width,height), 255)
# #     imgplot = plt.imshow(img)
# #     plt.show()
# #     # print img
# # #     img.paste(piece)
# # #     path = os.path.join("cam%d_1%05d.tif" % (int(k+1),filenum))
# # #     img.save(path)
# # #     os.rename(path,os.path.join("cam%d.1%05d" % (int(k+1),filenum)))
            

In [ ]:
row = {"newimage": "train_57_1", "oldimage": "train_57", "class": "artisian_mining"}
metadata = metadata.append(row, ignore_index = True)

In [ ]:
img_resize = (128, 128) # The resize size of each image ex: (64, 64) or None to use the default image size
validation_split_size = 0.2

In [ ]:
preprocessor = AmazonPreprocessor(train_jpeg_dir, train_csv_file, test_jpeg_dir, test_jpeg_additional, 
                                  img_resize, validation_split_size)
preprocessor.init()